In [87]:
import pandas as pd
import html5lib 
import datetime as dt

luoghi = ["Tokyo", "Boston", "London","Berlin","Chicago","New_York_City"]
Marathons=[]

for luogo in luoghi: #Creo la lista di maratone necessaria per poter effettuare lo scraping
    for year in range(2013,2024):
        Marathons.append(f"{year}_{luogo}_Marathon")

Deleted_Marathons = ["2020_Boston_Marathon", "2020_Berlin_Marathon", "2020_New_York_City_Marathon", "2020_Chicago_Marathon","2020_Tokyo_Marathon", "2014_Boston_Marathon"] 
#In questa lista sono inserite alcune maratone non disputate o che presentavano problemi nello scraping

for race in Deleted_Marathons: #Rimuovo le corse non presenti
        Marathons.remove(race)
        
#print(Marathons)
        
for marathon in Marathons: #con questo ciclo scansiono tutte le pagine di wikipedia riferite alla varie maratone indicate nella lista Marathons
    url = f"https://en.wikipedia.org/wiki/{marathon}"
    tables= pd.read_html(url, attrs={'class':'wikitable'})
    
    df = pd.concat([pd.DataFrame(table) for table in tables[:2]]) #Limito l'importazione nel dataframe alle prime 2 tabelle, ovver quelle dei risultati maschili e femminili
    row1 = len(tables[0].index) #Valuta la lunghezza di entrambe le tabelle per poter inserire il numero adatto di voci "Gender"
    row2 = len(tables[1].index)
        
    df['Position'] = df.index + 1 #Poichè alcune posizioni contengono caratteri speciali definisco le posizioni come indice + 1 
    
    year = dt.datetime.strptime(marathon.split("_")[0], "%Y").year # Estrapolo l'anno della maratona dalla lista delle competizioni
    location = marathon.split("_")[1]  # Estrapolo il luogo della maratona dalla lista delle competizioni

    df.insert(0, 'Location', f"{location}") #Inserisco il luogo della maratona nella tabella
    df.insert(1, 'Year', f"{year}") #Inserisco l'anno della maratona nella tabella
    df.insert(2, 'Gender', ['Men']*row1 + ['Women']*row2) #Inserisco il sesso dell'atleta, utilizzando la variabile row per contare il numero esatto di atleti

    df['Time'] = df['Time'].str.replace(r'[^\d:]', '', regex=True) #Rimuovo eventuali caratteri non numerici dalla colonno "Time", ad escludione dei :
    
    df.to_csv(f'{marathon}.csv', sep=',', index=False) #Esporto il data frame in file csv separatore virgola